In [158]:
import csv
import os
from functools import partial
import json
import random

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.mplot3d import Axes3D

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)
from gerrychain.metrics import efficiency_gap, mean_median, polsby_popper
from gerrychain.proposals import recom, flip
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part, bipartition_tree

import tqdm

In [4]:
graph_path = "./VA_precincts/VA_precincts.shp"
jgraph = Graph.from_json("va_json.json")

In [5]:
df = gpd.read_file(graph_path)

def num_splits(partition):
    df["current"] = df[unique_label].map(dict(partition.assignment))
    splits = sum(df.groupby("precinct")["current"].nunique() > 1)
    return splits

In [6]:
def pop_dist_pct(partition):
    ideal_population = ideal_population = sum(partition["population"].values()) / len(
    partition)
    total_deviation = total_deviation = sum([abs(v - ideal_population) for v in partition['population'].values()])
    avg_dist = total_deviation/len(partition)
    return avg_dist/ideal_population

my_updaters = {
    "cut_edges": cut_edges,
    "population": updaters.Tally("TOTPOP", alias = "population"),
    "pop_dist_pct" : pop_dist_pct,
    "area": updaters.Tally("Area", alias = "area"),
    "perimeter": updaters.Tally("Perimeter", alias = "perimeter")
}

In [7]:
df["nBVAP"] = df["VAP"] - df["BVAP"]
df.round({"VAP": 1, "BVAP": 1, "nBVAP": 1})
df["G18DSEN"] = df["G18DSEN"].astype(float)
df["G18RSEN"] = df["G18RSEN"].astype(float)
df["G17DGOV"] = df["G17DGOV"].astype(float)
df["G17RGOV"] = df["G17RGOV"].astype(float)
df["G16DPRS"] = df["G16DPRS"].astype(float)
df["G16RPRS"] = df["G16RPRS"].astype(float)

In [8]:
num_elections = 4

election_names = [
    "G18SEN",
    "G17GOV",
    "G16PRS",
    "BVAPs"
]

election_columns = [
    [df["G18DSEN"], df["G18RSEN"]],
    [df["G17DGOV"], df["G17RGOV"]],
    [df["G16DPRS"], df["G16RPRS"]],
    [df["BVAP"], df["nBVAP"]],
]

elections = [
    Election(
        election_names[i],
        {"Democratic": election_columns[i][0], "Republican": election_columns[i][1]},
    )
    for i in range(num_elections)
]

election_updaters = {election.name: election for election in elections}

my_updaters.update(election_updaters)

In [10]:
num_dist = 100
initial_partition = Partition(jgraph, "HDIST_11", my_updaters)
pop = sum(initial_partition["population"].values())
random_plan = recursive_tree_part(jgraph, range(num_dist), pop/num_dist, "TOTPOP", 0.01, 1)
initial_partition_random = Partition(jgraph, random_plan, my_updaters)

In [11]:
ideal_population = pop / len(initial_partition)

#proposal = partial(
#    recom, pop_col="TOTPOP", pop_target=ideal_population, epsilon=0.1, node_repeats=5
#)

In [155]:
pop_col = "TOTPOP"
pop = sum(initial_partition["population"].values())
pop_target = pop/num_dist
epsilon = 0.02
node_repeats = 5

In [174]:
def bvap_proposal(partition, pop_col, pop_target, epsilon, node_repeats, p):
    if random.random() < p:
        bvap_dict = {partition["BVAPs"].percents("Democratic")[i]:i for i in range(num_dist)}
        bvap_sort = sorted(list(partition["BVAPs"].percents("Democratic")))
        dist_choice = bvap_dict[bvap_sort[87]]
        dist_choice_nodes = []
        for x in range(initial_partition.graph.number_of_nodes()):
            if partition.assignment[x] == str(dist_choice):
                dist_choice_nodes.append(x)
        b_nodes = {x[0] for x in partition["cut_edges"]}.union(
            {x[1] for x in partition["cut_edges"]})
        node_poss = set(dist_choice_nodes).intersection(b_nodes)
        node_choice = random.choice(list(node_poss))
        neighbor_assignments = list(set([partition.assignment[neighbor] for neighbor in partition.graph.neighbors(node_choice)]))
        neighbor_assignments.remove(partition.assignment[node_choice])
        parts_to_merge = (int(partition.assignment[node_choice]), int(random.choice(neighbor_assignments)))
        subgraph = partition.graph.subgraph(
            partition.parts[str(parts_to_merge[0])] | partition.parts[str(parts_to_merge[1])]
        )
        flips = recursive_tree_part(
            subgraph,
            parts_to_merge,
            pop_col=pop_col,
            pop_target=pop_target,
            epsilon=epsilon,
            node_repeats=node_repeats,
            )
        return partition.flip(flips)
    else:
        recom

In [175]:
bvap_proposal(initial_partition, pop_col, pop_target, epsilon, node_repeats, 0.1)

In [168]:
subgraph

<Graph [43 nodes, 81 edges]>

'13'